In [5]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 22.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which

In [44]:
!pip install gradio

In [6]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training
from datasets import load_dataset
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
from torch.utils.data import DataLoader
import os
from transformers import TextGenerationPipeline
import gradio as gr

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [8]:
model_name = "openai-community/gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto" if torch.cuda.is_available() else None
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [9]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = model.config.eos_token_id

In [10]:
dataset = load_dataset("tatsu-lab/alpaca")
print(f"Dataset loaded with {len(dataset['train'])} training examples")


README.md:   0%|          | 0.00/7.47k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


(…)-00000-of-00001-a09b74b3ef9c3b56.parquet:   0%|          | 0.00/24.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/52002 [00:00<?, ? examples/s]

Dataset loaded with 52002 training examples


In [11]:
def prepare_training_example(example, max_length=512):
    """
    Prepares a dataset example for instruction fine-tuning by tokenizing and
    properly formatting inputs and labels.

    Args:
        example (dict): Dictionary containing 'instruction', 'input', and 'output' keys
        max_length (int): Maximum sequence length

    Returns:
        dict: Processed example with input_ids, attention_mask, and labels
    """
    try:
        # Format the prompt and full text
        if example['input']:
            prompt = f"Instruction: {example['instruction']}\nInput: {example['input']}\nOutput:"
        else:
            # Handle cases where input is empty
            prompt = f"Instruction: {example['instruction']}\nOutput:"

        full_text = prompt + " " + example["output"]

        # Tokenize the full sequence
        tokenized = tokenizer(
            full_text,
            truncation=True,
            max_length=max_length,
            padding="max_length",
            return_attention_mask=True
        )

        # Tokenize just the prompt to find its length
        prompt_tokens = tokenizer(
            prompt,
            add_special_tokens=False
        )

        # Set up labels: -100 for prompt tokens (to be ignored by loss function)
        labels = tokenized["input_ids"].copy()
        prompt_length = len(prompt_tokens["input_ids"])

        # Set prompt tokens to -100 so they're ignored in loss calculation
        for i in range(prompt_length):
            if i < len(labels):
                labels[i] = -100

        # Also mask padding tokens in the labels
        for i in range(len(labels)):
            if tokenized["attention_mask"][i] == 0:  # This is a padding token
                labels[i] = -100

        tokenized["labels"] = labels
        return tokenized

    except Exception as e:
        print(f"Error processing example: {e}")

        return {
            "input_ids": [tokenizer.pad_token_id] * 2,
            "attention_mask": [0] * 2,
            "labels": [-100] * 2
        }

In [12]:
max_length = 512  # Adjust based on your needs and GPU memory
processed_dataset = dataset.map(
    lambda x: prepare_training_example(x, max_length=max_length),
    remove_columns=dataset["train"].column_names,
    desc="Processing dataset",
    num_proc=4
)

Processing dataset (num_proc=4):   0%|          | 0/52002 [00:00<?, ? examples/s]

In [13]:
def evaluate_model(model, tokenizer, test_examples, device="cuda", max_new_tokens=100):
    """
    Evaluates a model on a list of test examples.

    Args:
        model: The model to evaluate
        tokenizer: The tokenizer to use
        test_examples: List of dictionaries with 'instruction', 'input', and 'reference_output' keys
        device: Device to run inference on
        max_new_tokens: Maximum number of tokens to generate

    Returns:
        dict: Dictionary with generated responses and metrics
    """
    model.eval()
    results = []

    for example in test_examples:
        instruction = example['instruction']
        input_text = example.get('input', '')
        reference = example.get('reference_output', '')

        # Format the prompt based on whether input is provided
        if input_text:
            prompt = f"Instruction: {instruction}\nInput: {input_text}\nOutput:"
        else:
            prompt = f"Instruction: {instruction}\nOutput:"

        # Tokenize and move to device
        inputs = tokenizer(prompt, return_tensors="pt").to(device)

        # Generate response
        with torch.no_grad():
            outputs = model.generate(
                input_ids=inputs["input_ids"],
                attention_mask=inputs["attention_mask"],
                max_new_tokens=max_new_tokens,
                temperature=0.7,
                top_p=0.9,
                do_sample=True,
                pad_token_id=tokenizer.pad_token_id,
            )

        # Decode and extract only the response part
        full_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
        response = full_output[len(prompt):].strip()

        results.append({
            'instruction': instruction,
            'input': input_text,
            'generated_output': response,
            'reference_output': reference
        })

    return results

In [14]:
# Sample test examples
test_examples = [
    {
        'instruction': 'Write a short poem',
        'input': 'about artificial intelligence',
        'reference_output': 'In silicon dreams and neural might,\nA new mind awakens to the light.\nNot born of flesh but human thought,\nIntelligence that we have wrought.'
    },
    {
        "instruction": "Translate the following sentence to German",
        "input": "I am hungry",
        "reference_output": "Ich habe Hunger"
    },
    {
        'instruction': 'Summarize the main idea',
        'input': 'The Internet of Things (IoT) refers to the billions of physical devices around the world that are now connected to the internet, collecting and sharing data.',
        'reference_output': 'The Internet of Things (IoT) is a network of billions of internet-connected physical devices worldwide that collect and share data.'
    },
    {
        'instruction': 'Explain the concept of photosynthesis in simple terms',
        'input': '',
        'reference_output': 'Photosynthesis is how plants make their own food. They use sunlight, water, and carbon dioxide to create energy and oxygen. It\'s like plants cooking their meals using sunlight as the heat source.'
    },
]

In [15]:
# Function to run tests and display results
def run_model_evaluation(model_name="Base Model", save_results=False):
    print(f"\n===== {model_name} Evaluation =====")

    model.to(device)
    results = evaluate_model(model, tokenizer, test_examples, device)

    for i, result in enumerate(results):
        print(f"\nExample {i+1}:")
        print(f"Instruction: {result['instruction']}")
        if result['input']:
            print(f"Input: {result['input']}")
        print(f"\nGenerated output: {result['generated_output']}")
        if result['reference_output']:
            print(f"Reference output: {result['reference_output']}")
        print("-" * 50)

    if save_results:
        import json
        import os
        results_dir = "./evaluation_results"
        os.makedirs(results_dir, exist_ok=True)

        with open(f"{results_dir}/{model_name.replace(' ', '_').lower()}_results.json", "w") as f:
            json.dump(results, f, indent=2)
        print(f"Results saved to {results_dir}/{model_name.replace(' ', '_').lower()}_results.json")

    return results

In [16]:
# 1. Test before fine-tuning
print("\n\n=============== BEFORE FINE-TUNING EVALUATION ===============")
before_results = run_model_evaluation("Before Fine-tuning", save_results=True)



=============== BEFORE FINE-TUNING EVALUATION ===============

===== Before Fine-tuning Evaluation =====

Example 1:
Instruction: Write a short poem
Input: about artificial intelligence

Generated output: about artificial intelligence
Input: about artificial intelligence
Output: about artificial intelligence
Output: about artificial intelligence
Output: about artificial intelligence
Input: about artificial intelligence
Output: about artificial intelligence
Output: about artificial intelligence
Output: about artificial intelligence
Input: about artificial intelligence
Output: about artificial intelligence
Output: about artificial intelligence
Input: about artificial intelligence
Output: about artificial intelligence
Input: about artificial intelligence
Output: about artificial intelligence
Input: about artificial intelligence
Reference output: In silicon dreams and neural might,
A new mind awakens to the light.
Not born of flesh but human thought,
Intelligence that we have wrought.
--

In [17]:
# Configure LoRA
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    target_modules=["c_attn", "c_proj"],
    inference_mode=False
)

In [18]:
# Prepare model for LoRA fine-tuning
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()


trainable params: 811,008 || all params: 125,250,816 || trainable%: 0.6475


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1264: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [19]:
# Set up training arguments
output_dir = "./gpt2-alpaca-lora"
training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    warmup_steps=100,
    max_steps=1000,  # Adjust based on dataset size and needs
    learning_rate=2e-4,
    fp16=torch.cuda.is_available(),
    logging_steps=10,
    save_steps=200,
    save_total_limit=3,
    report_to="tensorboard",
    run_name="gpt2-alpaca-lora",
    remove_unused_columns=False,
)

In [20]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Not using masked language modeling
)

In [21]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=processed_dataset["train"],
    data_collator=data_collator,
)

# Train model
trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
10,3.287900
20,3.286000
30,3.097700
40,3.094900
50,2.981000
60,2.803600
70,2.647400
80,2.604100
90,2.645200
100,2.646700


Step,Training Loss
10,3.287900
20,3.286000
30,3.097700
40,3.094900
50,2.981000
60,2.803600
70,2.647400
80,2.604100
90,2.645200
100,2.646700


TrainOutput(global_step=1000, training_loss=2.4141362743377686, metrics={'train_runtime': 862.342, 'train_samples_per_second': 18.554, 'train_steps_per_second': 1.16, 'total_flos': 4220535177216000.0, 'train_loss': 2.4141362743377686, 'epoch': 0.30766864087377893})

In [30]:
trainer.save_model("./my_trained_chatbot_model")

In [27]:
print("\n\n=============== AFTER FINE-TUNING EVALUATION ===============")
after_results = run_model_evaluation("After Fine-tuning", save_results=True)



=============== AFTER FINE-TUNING EVALUATION ===============

===== After Fine-tuning Evaluation =====

Example 1:
Instruction: Write a short poem
Input: about artificial intelligence

Generated output: Artificial intelligence is a new form of information processing that allows human beings to understand complex information and process it in a way that is not possible in the past. It is used to solve complex problems such as human-computer interactions, data mining, and more. Artificial intelligence is also used to solve complex problems such as AI problems such as artificial intelligence and artificial intelligence-driven solutions. Artificial intelligence can also be used to automate tasks such as analyzing data, making decisions, and performing tasks such as diagnosis, diagnosis
Reference output: In silicon dreams and neural might,
A new mind awakens to the light.
Not born of flesh but human thought,
Intelligence that we have wrought.
----------------------------------------------

In [32]:
# Loading the trained model
def load_model(model_path):
    tokenizer = AutoTokenizer.from_pretrained("gpt2")  # Base tokenizer, adjust if needed
    model = AutoModelForCausalLM.from_pretrained(model_path)

    # If your tokenizer doesn't have a padding token, set it
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    return model, tokenizer

In [41]:
class ChatbotConversation:
    def __init__(self, model_path):
        self.model, self.tokenizer = load_model(model_path)
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model.to(self.device)
        self.pipeline = TextGenerationPipeline(
            model=self.model,
            tokenizer=self.tokenizer,
            device=0 if self.device == "cuda" else -1
        )
        self.conversation_history = []

    def reset(self):
        self.conversation_history = []
        return "Conversation has been reset."

    def format_prompt(self):
        if not self.conversation_history:
            return ""

        formatted_history = []
        for role, message in self.conversation_history:
            formatted_history.append(f"{role}: {message}")

        return "\n".join(formatted_history) + "\nAssistant:"

    def generate_response(self, user_input, max_length=100, temperature=0.7):
        # Add user input to history
        self.conversation_history.append(("User", user_input))

        # Format the prompt with conversation history
        prompt = self.format_prompt()

        # Generate response
        generated_text = self.pipeline(
            prompt,
            max_length=len(self.tokenizer.encode(prompt)) + max_length,
            do_sample=True,
            temperature=temperature,
            top_p=0.92,
            top_k=50,
            repetition_penalty=1.1,
            pad_token_id=self.tokenizer.eos_token_id
        )[0]["generated_text"]

        # Extract only the model's response
        response = generated_text[len(prompt):].strip()

        # Clean up the response if it contains start of a new turn
        if "User:" in response:
            response = response.split("User:")[0].strip()

        # Add the response to history
        self.conversation_history.append(("Assistant", response))

        return response

In [42]:
#Creating a Gradio interface
def create_chatbot_interface(model_path):
    chatbot = ChatbotConversation(model_path)

    def user_interaction(message, history):
        return chatbot.generate_response(message)

    def reset_conversation():
        return chatbot.reset()

    with gr.Blocks() as demo:
        gr.Markdown("# Your Fine-Tuned GPT-2 Chatbot")

        chatbot_interface = gr.ChatInterface(
            user_interaction,
            examples=[
                "Hello, how can you help me today?",
                "Tell me something interesting.",
                "What capabilities do you have?"
            ],
            title="Chat with the AI Assistant"
        )

        with gr.Row():
            reset_btn = gr.Button("Reset Conversation")
            reset_btn.click(reset_conversation, outputs=gr.Textbox())

    return demo

In [43]:
if __name__ == "__main__":

    MODEL_PATH = "./my_trained_chatbot_model"

    print(f"Loading model from {MODEL_PATH}...")
    demo = create_chatbot_interface(MODEL_PATH)
    print("Starting Gradio interface...")
    demo.launch(share=True)  #

Loading model from ./my_trained_chatbot_model...


Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:338: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Starting Gradio interface...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b3574167b91f66c429.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
